In [1]:
import anatomist.api as ana
from soma.qt_gui.qtThread import QtThreadCall
from soma.qt_gui.qt_backend import Qt

a = ana.Anatomist()

from soma import aims
import pandas as pd
import numpy as np
import os

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


create qapp
done
Starting Anatomist.....
config file : /casa/home/.anatomist/config/settings.cfg
PyAnatomist Module present
PythonLauncher::runModules()
global modules: /casa/host/build/share/anatomist-5.2/python_plugins
home   modules: /casa/home/.anatomist/python_plugins
loading module simple_controls


existing QApplication: 0
QStandardPaths: XDG_RUNTIME_DIR not set, defaulting to '/tmp/runtime-ad279118'


loading module save_resampled
loading module selection
loading module bsa_proba
loading module modelGraphs
loading module profilewindow
loading module ana_image_math
loading module paletteViewer
loading module foldsplit
loading module anacontrolmenu
loading module gradientpalette
loading module palettecontrols
loading module meshsplit
loading module volumepalettes
loading module gltf_io
loading module infowindow
loading module histogram
loading module measure
loading module statsplotwindow
loading module valuesplotwindow
all python modules loaded
Anatomist started.


#### To visualize specific 3D volumic sulci for specific subjects

In [2]:
dataset =  'hcp'
region = 'S.C.-sylv.' #"S.C.-sylv." "S.T.s."
side = 'R' #"L"

In [3]:
sample = [102816,
 103010]

In [4]:
volume=True
nb_columns=2
block = a.createWindowsBlock(nb_columns) # nb of columns
dic_windows = {}

referential1 = a.createReferential()

mm_skeleton_path = f'/neurospin/dico/data/deep_folding/current/datasets/{dataset}/crops/2mm/{region}/mask/{side}crops'
# dic_windows['Sulci_color']=a.loadObject('/casa/host/build/share/brainvisa-share-5.2/nomenclature/hierarchy/sulcal_root_colors.hie')
dic_windows['Sulci_color'] = os.path.join(aims.carto.Paths.shfjShared(), 'nomenclature',
                                          'hierarchy', 'sulcal_root_colors.hie')

for i, subject_id in enumerate(sample):
    volume_path = f"{mm_skeleton_path}/{subject_id}_cropped_skeleton.nii.gz"
    
    if os.path.isfile(volume_path):
        vol = aims.read(volume_path)
        
        dic_windows[f'a_vol{nb_columns*i}'] = a.toAObject(vol)
        #dic_windows[f'a_vol{i}'].setPalette(absoluteMode=True)
        dic_windows[f'rvol{nb_columns*i}'] = a.fusionObjects(objects=[dic_windows[f'a_vol{nb_columns*i}']], method='VolumeRenderingFusionMethod')
        dic_windows[f'rvol{nb_columns*i}'].releaseAppRef()
        dic_windows[f'rvol{nb_columns*i}'].assignReferential(referential1)
        dic_windows[f'wvr{nb_columns*i}'] = a.createWindow('3D', block=block) #geometry=[100+400*(i%3), 100+440*(i//3), 400, 400])
        dic_windows[f'wvr{nb_columns*i}'].addObjects(dic_windows[f'rvol{nb_columns*i}'])
    else:
        print(f"{volume_path} is not a correct path, or the .nii.gz doesn't exist")

    path_to_t1mri = f'/neurospin/dico/data/bv_databases/human/not_labeled/hcp/hcp/{subject_id}/t1mri/BL' # be careful HCP here
    white_matter_path = f'{path_to_t1mri}/default_analysis/segmentation/mesh/{subject_id}_{side}white.gii'
    sulci_path = f'{path_to_t1mri}/default_analysis/folds/3.1/{side}{subject_id}.arg'
    spam_labelled_sulci_path = f'{path_to_t1mri}/default_analysis/folds/3.1/spam_auto/{side}{subject_id}_spam_auto.arg'
    deep_labelled_sulci_path = f'{path_to_t1mri}/default_analysis/folds/3.1/deepcnn_auto/{side}{subject_id}_deepcnn_auto.arg'

    if os.path.isfile(white_matter_path):
        # To visualize the white matter for specific people
        dic_windows[f'white_{subject_id}'] = a.loadObject(white_matter_path)
        #dic_windows[f'white_{subject_id}'].loadReferentialFromHeader()
        dic_windows[f'white_{subject_id}'].assignReferential(referential1)
    else:
        print(f"{white_matter_path} is not a correct path, or the .white.gii doesn't exist")

    #if os.path.isfile(sulci_path):
        # To visualize the sulci for specific people
        #dic_windows[f'sulci_{subject_id}'] = a.loadObject(sulci_path)
        #dic_windows[f'sulci_{subject_id}'].loadReferentialFromHeader()
    #else:
        #print(f"{sulci_path} is not a correct path, or the .arg doesn't exist")
    
    if os.path.isfile(spam_labelled_sulci_path):
        # To visualize the annotated sulci for specific people
        dic_windows[f'sulci_labelled_{subject_id}'] = a.loadObject(spam_labelled_sulci_path)
        #dic_windows[f'sulci_labelled_{subject_id}'].loadReferentialFromHeader()
        dic_windows[f'sulci_labelled_{subject_id}'].assignReferential(referential1)
    else:
        print(f"{spam_labelled_sulci_path} is not a correct path, or the .arg doesn't exist")
        print("Automatic try with 'deepcnn_session_auto' instead of 'spam_session_auto'")
        if  os.path.isfile(deep_labelled_sulci_path):
            # To visualize the annotated sulci for specific people
            dic_windows[f'sulci_labelled_{subject_id}'] = a.loadObject(deep_labelled_sulci_path)
            #dic_windows[f'sulci_labelled_{subject_id}'].loadReferentialFromHeader()
            dic_windows[f'sulci_labelled_{subject_id}'].assignReferential(referential1)
    
    #dic_windows[f'wvr{nb_columns*i+1}'] = a.createWindow('3D', block=block)
    #dic_windows[f'wvr{nb_columns*i+1}'].addObjects([dic_windows2[f'white_{subject_id}'], dic_windows[f'sulci_{subject_id}']])
    dic_windows[f'wvr{nb_columns*i+1}'] = a.createWindow('3D', block=block)
    dic_windows[f'wvr{nb_columns*i+1}'].addObjects(dic_windows[f'white_{subject_id}'])
    dic_windows[f'wvr{nb_columns*i+1}'].addObjects(dic_windows[f'sulci_labelled_{subject_id}'])

Multitexturing present
function glActiveTexture found.
function glClientActiveTexture found.
function glBlendEquation found.
function glTexImage3D found.
function glMultiTexCoord3f found.
function glBindFramebuffer found.
function glBindRenderbuffer found.
function glFramebufferTexture2D found.
function glGenFramebuffers found.
function glGenRenderbuffers found.
function glFramebufferRenderbuffer found.
function glRenderbufferStorage found.
function glCheckFramebufferStatus found.
function glDeleteRenderbuffers found.
function glDeleteFramebuffers found.
Number of texture units: 4
function glUniform1f found.
function glUniform1i found.
function glUniform4fv found.
function glGetUniformLocation found.
function glMultTransposeMatrixf found.
function glAttachShader found.
function glDetachShader found.
function glCompileShader found.
function glCreateProgram found.
function glCreateShader found.
function glDeleteProgram found.
function glDeleteShader found.
function glGetProgramiv found.


Position : 42.0281, 61.0887, 110.176, 0
no position could be read at 357, 277
no position could be read at 319, 389
no position could be read at 382, 416
Position : 36.4796, 97.2007, 83.2995, 0


QLayout: Attempting to add QLayout "" to QWidget "", which already has a layout


Position : 29.0139, 118.615, 114.35, 0
no position could be read at 27, 283
Position : 85.118, 177.049, 91.9312, 0
Position : 89.8196, 170.439, 104.416, 0


In [ ]:
dic_windows

{'Sulci_color': <anatomist.cpp.anatomist.AObject object at 0x70541a1ed750>,
 'a_vol0': <anatomist.cpp.anatomist.SliceableObject object at 0x70541a168f70>,
 'rvol0': <anatomist.cpp.anatomist.MObject object at 0x70541a1ed870>,
 'wvr0': <anatomist.cpp.weak_shared_ptr_AWindow object at 0x70540f525090>,
 'white_102816': <anatomist.cpp.anatomist.ASurface_3 object at 0x70541a169090>,
 'sulci_labelled_102816': <anatomist.cpp.anatomist.AGraph object at 0x70540eb83370>,
 'wvr1': <anatomist.cpp.weak_shared_ptr_AWindow object at 0x70540f524f70>,
 'a_vol2': <anatomist.cpp.anatomist.SliceableObject object at 0x70541a1ed7e0>,
 'rvol2': <anatomist.cpp.anatomist.MObject object at 0x70540ebaa170>,
 'wvr2': <anatomist.cpp.weak_shared_ptr_AWindow object at 0x70540ebb1360>,
 'white_103010': <anatomist.cpp.anatomist.ASurface_3 object at 0x70540ebb17e0>,
 'sulci_labelled_103010': <anatomist.cpp.anatomist.AGraph object at 0x70540ebb1c60>,
 'wvr3': <anatomist.cpp.weak_shared_ptr_AWindow object at 0x70540ebb15a